## Real time face detection and antispoofing

This code implements a real-time face anti-spoofing system using face detection and machine learning techniques. It combines several modules, including OpenCV for video capture, FaceNet-PyTorch (MTCNN) for face detection, and the pre-trained MobileNetV2 model (we trained the last layer of the model on the datasets MSU-MFSD Photos, NUAA and on their combination) to classify faces as "real" or "spoof".

**1. Face Detection**: the code uses the MTCNN module from the facenet_pytorch library to detect faces in real-time from a webcam feed. Once a face is detected, the corresponding region is cropped from the video frame for further analysis.

**2. Image Preprocessing**: the detected face is preprocessed before feeding it to the deep learning model. A specific technique, Local Binary Patterns (LBP), is applied optionally to enhance the texture features of the face image, which can help in spoofing detection.

**3. Model Loading and Prediction**: the MobileNetV2 model is loaded, and its final layer is modified to output a binary classification for spoof detection. The preprocessed image is passed through the model, which outputs a score that determines whether the face is classified as "real" or "fake."

**4. Real-Time Detection Loop**: a continuous loop captures frames from the webcam, detects faces, and performs spoofing detection on each detected face. The result is visualized by drawing a rectangle around the detected face and labeling it as either "Real" (green) or "Fake" (red). The loop continues until the user interrupts the program by pressing 'q'.

In [3]:
import os
import cv2
import warnings
import torch
from facenet_pytorch import MTCNN
import numpy as np
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

from skimage.feature import local_binary_pattern

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

warnings.filterwarnings("ignore")

cuda


In [5]:
# Set parameters for the Local Binary Pattern (LBP) feature extraction
radius = 1 # Radius for LBP calculation
n_points = 8 * radius # Number of points for LBP in a circular neighborhood

# Function to apply LBP on a single channel (R, G or B) of the image
def apply_lbp(channel):
    return local_binary_pattern(channel, n_points, radius, method='uniform')

# Function to apply LBP transformation to the entire RGB image
def lbp_transform(image):
    # Apply LBP to each color channel (R, G, B)
    lbp_r = apply_lbp(image[:, :, 0])
    lbp_g = apply_lbp(image[:, :, 1])
    lbp_b = apply_lbp(image[:, :, 2])
    # Normalize each channel to a range of 0-255 for display purposes
    lbp_r_normalized = cv2.normalize(lbp_r, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX).astype(np.uint8)
    lbp_g_normalized = cv2.normalize(lbp_g, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX).astype(np.uint8)
    lbp_b_normalized = cv2.normalize(lbp_b, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX).astype(np.uint8)
    # Merge the LBP channels back into a color image
    lbp_color = cv2.merge((lbp_r_normalized, lbp_g_normalized, lbp_b_normalized))
    return lbp_color

In [6]:
# Function to load pre-trained MobileNetV2 model (trained on MSU-MFSD Photos, NUAA and on their combination) and modify it for binary classification
def load_trained_mobilenet_model(model_path, device):
    loaded_model = models.mobilenet_v2(pretrained=False)
    loaded_model.classifier[1] = nn.Linear(loaded_model.classifier[1].in_features, 1)      
    loaded_model.load_state_dict(torch.load(model_path)) 
    loaded_model = loaded_model.to(device)
    loaded_model.eval()
    return loaded_model

In [7]:
def preprocess_image(image, target_size):
    transform = transforms.Compose([
        transforms.ToPILImage(), # Convert image to PIL format
        transforms.Resize(target_size), # Resize the image to the target size (224x224)
        transforms.ToTensor(), # Convert image to tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Normalize
    ])
    image = transform(image) # Apply transformations to the image
    image = image.unsqueeze(0)
    return image

In [8]:
# Function to predict whether a face is "real" or "fake" using the loaded model
def detect_real_or_fake(image, model, target_size):
    model.eval()
    with torch.no_grad(): # Disable gradient computation for faster inference
        processed_image = preprocess_image(image, target_size).to(device) # Preprocess and move image to the device
        output = model(processed_image) # Run the image through the model
        prediction = torch.sigmoid(output).item() # Get the prediction (applying sigmoid for binary classification)
        return "Fake" if prediction > 0.5 else "Real" # Classify based on the output value with respect to the threshold

In [9]:
# Main function to run face detection and anti-spoofing in real-time
def localize_and_detect_spoof_real_time(loaded_model, additional_preprocessing=None):
    # Initialize MTCNN for face detection
    mtcnn = MTCNN(keep_all=False)
    IMG_SIZE = (224, 224) # Target size for the input images to the model
    cap = cv2.VideoCapture(0 + cv2.CAP_DSHOW)
    face_cropped_window_open = False 
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # Convert the frame to RGB format for face detection
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_pil = Image.fromarray(frame_rgb)
        boxes, _ = mtcnn.detect(frame_pil) # Detect faces in the frame
        
        if boxes is not None: # If any face is detected
            for box in boxes:
                x1, y1, x2, y2 = [int(coord) for coord in box] # Get coordinates of the bounding box
                x1, y1 = max(x1, 0), max(y1, 0)
                x2, y2 = min(x2, frame.shape[1]), min(y2, frame.shape[0])
                
                if x1 < x2 and y1 < y2:
                    face = frame[y1:y2, x1:x2] # Crop the face region from the frame
    
                    if face.size > 0:
                        face = cv2.resize(face, IMG_SIZE)  # Resize the face to match model input size (224x224)
                        if additional_preprocessing == 'lbp':
                            face = lbp_transform(face) # Apply LBP if specified
    
                        result = detect_real_or_fake(face, loaded_model, IMG_SIZE) # Predict whether the face is real or fake
                        # Draw a rectangle around the face with color based on the result (green for real, red for fake)
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0) if result == "Real" else (0, 0, 255), 2)
                        cv2.putText(frame, result, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0) if result == "Real" else (0, 0, 255), 2)
    
                        if not face_cropped_window_open:
                            face_cropped_window_open = True
                        cv2.imshow('Face cropped', face)
                    else:
                        if face_cropped_window_open:
                            cv2.destroyWindow('Face cropped')
                            face_cropped_window_open = False
        else:
            if face_cropped_window_open:
                cv2.destroyWindow('Face cropped')
                face_cropped_window_open = False
    
        cv2.imshow('Face Anti-Spoofing', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [10]:
loaded_model = load_trained_mobilenet_model(r"C:\Users\Asus\Desktop\Progetto_BS_Frabotta_Ferrone\Evaluation\Face_antispoofing\Models\mobilenet_v2_combined_model.pth", device)

In [11]:
localize_and_detect_spoof_real_time(loaded_model, additional_preprocessing=None)